In [1]:
import pandas as pd
import numpy as np
import schedule
import time
import os
""" import openpyxl 
from openpyxl import  load_workbook """

#creamos la funcion MonitoringDueAcc()
def MonitoringDueAcc():
    try:
        #leemos el archivo de excel o csv almacenado dentro de una variable
        df = pd.read_csv("sample_data_intw.csv")        #encoding="utf-8", encoding_errors="replace"

        #convertir las fechas en formato datetime 
        df["last_rech_date_ma"] = pd.to_datetime(df["last_rech_date_ma"], errors="coerce")
        df["pdate"] = pd.to_datetime(df["pdate"], errors="coerce", dayfirst=True)

        #filtramos por la columna que tiene informacion de 
        deliquency_over30 = df[df["daily_decr30"] > 30]

        sol = deliquency_over30[[ "msisdn", "daily_decr30", "rental30",]].head(100)
        print(f"Data processed successfully.")
        return sol
    except Exception as e:
        print(f"An error has occurred: {e}")
        return pd.DataFrame()


def GenerateReportAuto():
    try:
        #el mensaje que se mostrara cuando se ejecute la funcion
        print("Report Generation In Process...") 
        sol = MonitoringDueAcc() #llamamos a la funcion MonitoringDueAcc()
        if not sol.empty: #si la variable sol no esta vacia
            #guardamos el archivo en formato excel
            output_file = os.path.join(os.getcwd(), "MonitoringDueAcc.xlsx") 
            sol.to_excel(output_file, index=False)
            #imprimimos el mensaje de que se genero el reporte exitosamente
            print(f"Report Generated Successfully at {output_file}") 
        else: #si la variable sol esta vacia mostrara el mensaje de que fallo la generacion del reporte
            print("Report Generation Failed.")
    except Exception as e: #si ocurre un error mostrara el mensaje de que fallo la generacion del reporte
        print(f"An error has occured: {e}")

#definimos la funcion GenerateReportAuto() que se ejecut
schedule.every().friday.at(f"17:15").do(GenerateReportAuto)
print(f"Waiting for the report to be Generated at 5:15 PM on Friday...")

#mantenemos el programa en ejecucion
while True:
    schedule.run_pending()
    time.sleep(2)




Waiting for the report to be Generated at 5:07 PM on Friday...
Report Generation In Process...
Data processed successfully.
Report Generated Successfully at c:\Users\nandy\OneDrive\Escritorio\PythonTest\Collect Analysis\MonitoringDueAcc.xlsx


KeyboardInterrupt: 

In [ ]:
#Generacion de reportes automaticamente
import schedule
import time
import os


def GenerateReportAuto():
    try:
        print("Report Generation In Process...")
        result = MonitoringDueAcc() 
        if not result.empty:
            output_file = os.path.join(os.getcwd(), "MonitoringDueAcc.xlsx") 
            result.to_excel(output_file, index=False)
            print(f"Report Generated Successfully at {output_file}") 
        else:
            print("Report Generation Failed.")
    except Exception as e:
        print(f"An error has occured: {e}")

schedule.every().friday.at(f"17:07").do(GenerateReportAuto)
print(f"Waiting for the report to be Generated at 5:07 PM on Friday...")

while True:
    schedule.run_pending()
    time.sleep(2)

In [ ]:
#Para contar a partir de una tabla frecuencia la cantidad de los numeros de telefono
df = pd.read_csv("sample_data_intw.csv")
frequency = df["msisdn"].value_counts().head(10)
frequency

In [ ]:
#para agrupar por msisdn y sumar los valores de daily_decr30
goup_by = df.groupby(["msisdn"])[["daily_decr90"]].sum().head(10)
goup_by